In [1]:
import pandas as pd
import numpy as np
import polars as pl
import pyarrow.parquet as pq

import tensorflow as tf
import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam


import gc


from utils import reduce_memory, config

CONFIG = config.CONFIG
columns = CONFIG.feature_names + CONFIG.exogeneous_features + CONFIG.lag_features

tf.config.experimental.list_physical_devices("GPU")
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
class R2Metric(tf.keras.metrics.Metric):
    def __init__(self, name="r2", **kwargs):
        super(R2Metric, self).__init__(name=name, **kwargs)
        self.squared_error = self.add_weight(name="squared_error", initializer="zeros")
        self.total_error = self.add_weight(name="total_error", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Calculate squared error
        y_true = tf.cast(tf.squeeze(y_true), dtype=tf.float32)
        y_pred = tf.cast(y_pred, dtype=tf.float32)
        sample_weight = tf.cast(tf.squeeze(sample_weight), dtype=tf.float32)
        squared_error = (y_pred - y_true) ** 2
        total_error = y_true**2

        # Update the total squared error, total error, and total weight
        self.squared_error.assign_add(tf.reduce_sum(squared_error * sample_weight))
        self.total_error.assign_add(tf.reduce_sum(total_error * sample_weight))

    def result(self):
        # Compute R²: 1 - (squared_error / total_error)
        return 1 - (self.squared_error / (self.total_error + 1e-38))

    def reset_state(self):
        # Reset all metrics at the end of each epoch
        self.squared_error.assign(0)
        self.total_error.assign(0)

In [3]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="silu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res


def build_model(
    input_shape,
    head_size=128,
    num_heads=8,
    ff_dim=128,
    num_transformer_blocks=2,
    mlp_units=[128],
    mlp_dropout=0.1,
    dropout=0.25,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D()(x)
    for units in mlp_units:
        x = layers.Dense(units, activation="silu")(x)
        x = layers.Dropout(mlp_dropout)(x)

    outputs = layers.Dense(1, activation="tanh")(x)
    outputs = Lambda(lambda x: x * 5)(outputs)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=1e-3, decay=5e-4),
        loss="mean_squared_error",
        weighted_metrics=[R2Metric()],
    )
    return model

In [4]:
valid_features_file_path = f"{CONFIG.main}/data/training_data_w_lag/X_valid.parquet"
valid_labels_file_path = f"{CONFIG.main}/data/training_data_w_lag/y_valid.parquet"
valid_weights_file_path = f"{CONFIG.main}/data/training_data_w_lag/w_valid.parquet"

In [5]:
batch_size = 128
seq_len = 256
feature_len = len(columns)
features_shape = (seq_len, feature_len)
epochs = 2_000

callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

features_batch = reduce_memory.reduce_mem_usage(
    pd.read_parquet(valid_features_file_path, columns=columns).fillna(0)
).values

labels_batch = reduce_memory.reduce_mem_usage(
    pd.read_parquet(valid_labels_file_path).fillna(0)
).values.squeeze()
weights_batch = reduce_memory.reduce_mem_usage(
    pd.read_parquet(valid_weights_file_path).fillna(0)
).values.squeeze()

original_size = features_batch.size
target_elements = seq_len * feature_len
if original_size % target_elements != 0:
    num_elements_to_keep = int(
        (original_size // target_elements) * target_elements / feature_len
    )
    features_batch = features_batch[:num_elements_to_keep]
    labels_batch = labels_batch[:num_elements_to_keep]
    weights_batch = weights_batch[:num_elements_to_keep]

features_batch = features_batch.reshape(-1, seq_len, feature_len)
labels_batch = labels_batch.reshape(-1, seq_len, 1)
weights_batch = weights_batch.reshape(-1, seq_len, 1)

with tf.device("/CPU:0"):
    valid_dataset = tf.data.Dataset.from_tensor_slices(
        (features_batch, labels_batch, weights_batch)
    )
    valid_dataset = valid_dataset.batch(batch_size)

Memory usage of dataframe is 730.06 MB
Memory usage after optimization is: 645.01 MB
Decreased by 11.65%
Memory usage of dataframe is 7.09 MB
Memory usage after optimization is: 7.09 MB
Decreased by 0.00%
Memory usage of dataframe is 7.09 MB
Memory usage after optimization is: 7.09 MB
Decreased by 0.00%


In [ ]:
for fold in range(CONFIG.N_fold):
    model = build_model(input_shape=features_shape)

    train_features_file_path = (
        f"{CONFIG.main}/data/training_data_w_lag/X_train_{fold}.parquet"
    )
    train_labels_file_path = (
        f"{CONFIG.main}/data/training_data_w_lag/y_train_{fold}.parquet"
    )
    train_weights_file_path = (
        f"{CONFIG.main}/data/training_data_w_lag/w_train_{fold}.parquet"
    )

    # Create the TensorFlow Dataset
    train_features_file = pq.ParquetFile(train_features_file_path)
    train_labels_file = pq.ParquetFile(train_labels_file_path)
    train_weights_file = pq.ParquetFile(train_weights_file_path)

    train_row_group = train_features_file.num_row_groups

    step = int(train_row_group / 3)

    for i in range(0, train_row_group, step):
        print(i)
        batch_end = min(i + step, train_row_group)
        features_batch = reduce_memory.reduce_mem_usage(
            train_features_file.read_row_groups(
                [i for i in range(i, batch_end)], columns=columns
            )
            .to_pandas()
            .fillna(0)
        ).values

        original_size = features_batch.size
        target_elements = seq_len * feature_len

        labels_batch = reduce_memory.reduce_mem_usage(
            train_labels_file.read_row_groups([i for i in range(i, batch_end)])
            .to_pandas()
            .fillna(0)
        ).values.squeeze()
        weights_batch = reduce_memory.reduce_mem_usage(
            train_weights_file.read_row_groups([i for i in range(i, batch_end)])
            .to_pandas()
            .fillna(0)
        ).values.squeeze()

        if original_size % target_elements != 0:
            num_elements_to_keep = int(
                (original_size // target_elements) * target_elements / feature_len
            )
            features_batch = features_batch[:num_elements_to_keep]
            labels_batch = labels_batch[:num_elements_to_keep]
            weights_batch = weights_batch[:num_elements_to_keep]

        features_batch = features_batch.reshape(-1, seq_len, feature_len)
        labels_batch = labels_batch.reshape(-1, seq_len, 1)
        weights_batch = weights_batch.reshape(-1, seq_len, 1)

        with tf.device("/CPU:0"):
            train_dataset = tf.data.Dataset.from_tensor_slices(
                (features_batch, labels_batch, weights_batch)
            )
            train_dataset = train_dataset.batch(batch_size).prefetch(
                tf.data.experimental.AUTOTUNE
            )

        with tf.device("/GPU:0"):
            model.fit(
                train_dataset,
                epochs=epochs,
                validation_data=valid_dataset,
                callbacks=[callback],
            )

        del train_dataset

    tf.keras.models.save_model(
        model, f"{CONFIG.main}/Models_impt/NN/NN_attn_{fold+1}.h5"
    )

0
Memory usage of dataframe is 2266.00 MB
Memory usage after optimization is: 2002.00 MB
Decreased by 11.65%
Memory usage of dataframe is 22.00 MB
Memory usage after optimization is: 22.00 MB
Decreased by 0.00%
Memory usage of dataframe is 22.00 MB
Memory usage after optimization is: 22.00 MB
Decreased by 0.00%
Epoch 1/2000
352/352 [==============================] - 107s 294ms/step - loss: 35.3350 - r2: -25.1235 - val_loss: 12.0858 - val_r2: -7.1653
Epoch 2/2000
352/352 [==============================] - 103s 293ms/step - loss: 3.5426 - r2: -1.5212 - val_loss: 1.5628 - val_r2: -0.0042
Epoch 3/2000
352/352 [==============================] - 102s 289ms/step - loss: 1.5468 - r2: -0.0390 - val_loss: 1.5585 - val_r2: -0.0011
Epoch 4/2000
352/352 [==============================] - 102s 289ms/step - loss: 1.5182 - r2: -0.0172 - val_loss: 1.5576 - val_r2: -4.0078e-04
Epoch 5/2000
352/352 [==============================] - 101s 288ms/step - loss: 1.5066 - r2: -0.0085 - val_loss: 1.5581 - val_r2